# Preliminaries

In [ ]:
import random
import numpy as np
import seaborn
import pandas as pd
import sys
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
import tigramite
from tigramite import data_processing as pp
from tigramite.toymodels import structural_causal_processes as toys

from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.lpcmci import LPCMCI

from tigramite.independence_tests.parcorr import ParCorr
from tigramite.independence_tests.robust_parcorr import RobustParCorr
from tigramite.independence_tests.parcorr_wls import ParCorrWLS 
#from tigramite.independence_tests.gpdc import GPDC
from tigramite.independence_tests.cmiknn import CMIknn
from tigramite.independence_tests.cmisymb import CMIsymb
from tigramite.independence_tests.gsquared import Gsquared
from tigramite.independence_tests.regressionCI import RegressionCI

In [ ]:
sys.path.append("../causalFS/CMI_FS")
from mixedRVMI import MIEstimate,estimateAllMI,CMIEstimate
from feature_selection import backwardFeatureSelection,forwardFeatureSelection
sys.path.append("../causalFS/CMI")
from FS import backwardFeatureSelection, forwardFeatureSelection
from TE_FS import TE_backwardFeatureSelection, TE_forwardFeatureSelection

sys.path.append("../causalFS/data")
from dataset_generation import generate_dataset


In [ ]:
def run_PCMCI(df,tau=0,cond_test=ParCorr()):    

    dataframe = pp.DataFrame(df.values)
    
    cond_ind_test = cond_test
    pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
    results = pcmci.run_pcmciplus(tau_min=1, tau_max=tau)
    #results = pcmci.run_pcmci(tau_max=tau, pc_alpha=0.01)

    pcmci.print_significant_links(p_matrix=results['p_matrix'],val_matrix=results['val_matrix'])
    print(results['p_matrix'])
    
    tp.plot_graph(
    val_matrix=results['val_matrix'],
    graph=results['graph'],
    var_names=list(df.columns),
    link_colorbar_label='cross-MCI',
    node_colorbar_label='auto-MCI',
    show_autodependency_lags=False
    )
    plt.show()
    
    tp.plot_time_series_graph(
    figsize=(6, 4),
    val_matrix=results['val_matrix'],
    graph=results['graph'],
    var_names=list(df.columns),
    link_colorbar_label='MCI',
    )
    plt.show()
    
    return results['p_matrix']
    

In [ ]:
def run_fullCI(df,tau=0,cond_test=ParCorr()):    

    dataframe = pp.DataFrame(df.values)
    
    cond_ind_test = cond_test
    pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
    #results = pcmci.run_pcmciplus(tau_max=tau, pc_alpha=0.01)
    #results = pcmci.run_pcmci(tau_max=tau, pc_alpha=0.01)
    results = pcmci.run_fullci(tau_min=1, tau_max=tau)

    pcmci.print_significant_links(p_matrix=results['p_matrix'],val_matrix=results['val_matrix'])
    
    tp.plot_graph(
    val_matrix=results['val_matrix'],
    graph=results['graph'],
    var_names=list(df.columns),
    link_colorbar_label='cross-MCI',
    node_colorbar_label='auto-MCI',
    show_autodependency_lags=False
    )
    plt.show()
    
    tp.plot_time_series_graph(
    figsize=(6, 4),
    val_matrix=results['val_matrix'],
    graph=results['graph'],
    var_names=list(df.columns),
    link_colorbar_label='MCI',
    )
    plt.show()
    
    return results['p_matrix']
    

In [ ]:
def concatenate_past(x,lag):
    lag_x = x[lag-1:-1].reshape(-1,1)
    if lag<=1: return lag_x
        
    for i in range(lag-1):
        lag_x = np.concatenate((lag_x, x[lag-i-2:-i-2].reshape(-1,1)),axis=1)
        
    return lag_x

In [ ]:
def evaluate_CI(x):
    x_mean = np.mean(x)
    x_std = np.std(x)
    return [x_mean-1.96*x_std/np.sqrt(len(x)),x_mean+1.96*x_std/np.sqrt(len(x))]

# Linear, 3 variables

### Preliminaries

In [ ]:
def generate_dataset_3dim(data_for_X0, c, d, n, seed=0, noise=0.1):
    
    data_for_X0 = pd.read_csv('../causalFS/data/sampleX0.csv')
    
    rng = np.random.default_rng(seed=seed)
    random.seed(seed)
    np.random.seed(seed)
    
    x0 = data_for_X0.iloc[:n+2].values
    x0 +=  np.random.randn(len(x0),1) * noise
    plt.plot(x0[2:],'g')
    
    y = list(np.zeros(n+2))
    for i in range(n):
        next_val = y[i]*c[0] + x0[i]*c[1] + np.random.randn() * noise
        y[i+2] = next_val[0]
    plt.plot(y[2:],'b')
    
    x1 = list(np.zeros(n+2))
    for i in range(n):
        next_val = y[i+1]*d + np.random.randn() * noise
        x1[i+2] = next_val
    plt.plot(x1[2:],'r')
    plt.show()
    
    x0 = np.array(x0)[2:].reshape(-1,1)
    x1 = np.array(x1)[2:].reshape(-1,1)
    y = np.array(y)[2:].reshape(-1,1)
    print(x0.shape, x1.shape, y.shape)

    x = np.concatenate((x0,x1),axis=1)
    
    return x,y

In [ ]:
data_for_X0 = pd.read_csv('../causalFS/data/sampleX0.csv')
plt.plot(data_for_X0.iloc[:300].values)

In [ ]:
x,y = generate_dataset_3dim('', [0.75,-0.65], 0.7, 300, seed=0, noise=0.1)

In [ ]:
def compute_r2(inputs,y,tau=2,n=300):
    features = concatenate_past(inputs[0],tau)
    if len(inputs)>1:
        for i in range(len(inputs)-1):
            actual = concatenate_past(inputs[i+1],tau)
            features = np.concatenate((actual,features),axis=1)

    y_target = y[tau:]
    
    split = round(n*0.66)
    
    reg = LinearRegression().fit(features[:split,:], y_target[:split].reshape(-1,1))
    score = reg.score(features[split:,:], y_target[split:].reshape(-1,1))
    print(f'R2 score: {score}\n')
    
    return score

In [ ]:
def experiment_3dim_fixedCoeff(data_for_X0, c, d, n, noise=0.1, n_reps=10, tau=2):
    correct_CMI_forw = 0
    wrong_CMI_forw = 0
    correct_CMI_back = 0
    wrong_CMI_back = 0
    correct_TE_forw = 0
    wrong_TE_forw = 0
    correct_TE_back = 0
    wrong_TE_back = 0
    correct_fullCI = 0
    correct_PCMCI = 0
    wrong_fullCI = 0
    wrong_PCMCI = 0
    
    tot_links = 0
    correct_links = set([0,2])
    
    res = {
                "delta" : [], # list with all deltas
                "numSelected" : [], #
                "selectedFeatures" : [] 
            #    "accuracy" : [] # list of scores associated with the reduced problem
            }
    
    for seed in range(n_reps):
        
        ### generate dataset
        x,y = generate_dataset_3dim(data_for_X0=data_for_X0, c=c, d=d, n=n, seed=seed, noise=noise)
        
        k = round(n/15)
        
        sel = forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_forw_sel = set(sel)
        
        sel = TE_forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        TE_forw_sel = set(sel)
        
        sel = backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_back_sel = set(sel)
    
        sel = TE_backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        TE_back_sel = set(sel)
        
        p = run_fullCI(pd.DataFrame(np.concatenate((x,y),axis=1), columns = ['x0','x1','y']),tau=tau,cond_test=ParCorr())
        sel = []
        for i in [0,1,2]:
            if min(p[i,2,:])<0.05:
                sel.append(i)
        full_CI_sel = set(sel)
        print(full_CI_sel)
        
        p = run_PCMCI(pd.DataFrame(np.concatenate((x,y),axis=1), columns = ['x0','x1','y']),tau=tau,cond_test=ParCorr())
        sel = []
        for i in [0,1,2]:
            if min(p[i,2,:])<0.05:
                sel.append(i)
        PCMCI_sel = set(sel)
        print(PCMCI_sel)
    
        print('\n\n')
        
        correct_CMI_forw += len([found for found in CMI_forw_sel if found in correct_links])
        wrong_CMI_forw += len(CMI_forw_sel) - len([found for found in CMI_forw_sel if found in correct_links])
        correct_CMI_back += len([found for found in CMI_back_sel if found in correct_links])
        wrong_CMI_back += len(CMI_back_sel) - len([found for found in CMI_back_sel if found in correct_links])
        correct_TE_forw += len([found for found in TE_forw_sel if found in correct_links])
        wrong_TE_forw += len(TE_forw_sel) - len([found for found in TE_forw_sel if found in correct_links])
        correct_TE_back += len([found for found in TE_back_sel if found in correct_links])
        wrong_TE_back += len(TE_back_sel) - len([found for found in TE_back_sel if found in correct_links])
        correct_fullCI += len([found for found in full_CI_sel if found in correct_links])
        wrong_fullCI += len(full_CI_sel) - len([found for found in full_CI_sel if found in correct_links])
        correct_PCMCI += len([found for found in PCMCI_sel if found in correct_links])
        wrong_PCMCI += len(PCMCI_sel) - len([found for found in PCMCI_sel if found in correct_links])
        tot_links += len(correct_links)
        
        #compute_r2([x[:,0],x[:,1]],y,tau=2,n=300)
    
    corrects = [correct_CMI_forw, correct_CMI_back, correct_TE_forw, correct_TE_back,correct_fullCI,correct_PCMCI]
    wrongs = [wrong_CMI_forw, wrong_CMI_back, wrong_TE_forw, wrong_TE_back, wrong_fullCI, wrong_PCMCI]
    print(corrects)
    print(wrongs)  
    print(tot_links)

    return corrects, wrongs, tot_links


### single example

In [ ]:
corrects, wrongs, tot_links = experiment_3dim_fixedCoeff(data_for_X0, c=[0.75,-0.65], d=0.70, n=300, noise=0.1, n_reps=10, tau=2)



In [ ]:
corrects, wrongs, tot_links

### variable tau

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []
for tau in [2,3,4,5]:
    corr, wron, tot_li = experiment_3dim_fixedCoeff(data_for_X0, c=[0.75,-0.65], d=0.70, n=300, noise=0.1, n_reps=10, tau=tau)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)

In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### increasing noise

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []
for noise in [0.1,0.3,0.5,0.7,0.9]:
    corr, wron, tot_li = experiment_3dim_fixedCoeff(data_for_X0, c=[0.75,-0.65], d=0.70, n=300, noise=noise, n_reps=10, tau=2)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)


In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### variable number of samples

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []
for n in [100,200,300,400,500]:
    corr, wron, tot_li = experiment_3dim_fixedCoeff(data_for_X0, c=[0.75,-0.65], d=0.70, n=n, noise=0.1, n_reps=10, tau=2)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)


In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### variable coefficients

In [ ]:
rng = np.random.default_rng(seed=0)
random.seed(0)
np.random.seed(0)
cc = []
dd = []
while len(cc)<6:
    comb = [rng.uniform(-1,1),rng.uniform(-1,1)]
    if (((comb[0]<=-0.5) | (comb[0]>=0.5)) & ((comb[1]<=-0.5) | (comb[1]>=0.5))):
        cc.append(comb)
        
while len(dd)<6:
    val = rng.uniform(-1,1)
    if (((val<=-0.5) | (val>=0.5))):
        dd.append(val)
    
print(cc,dd)

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []

for i in range(11):
    corr, wron, tot_li = experiment_3dim_fixedCoeff('', c=cc[i], d=dd[i], n=300, noise=0.1, n_reps=10, tau=2)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)


In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

# Same, 5 variables

### Preliminaries

In [ ]:
def experiment_5dim_fixedCoeff(data_for_X1, data_for_X3, a, b, c, n, noise=0.1, n_reps=10, tau=2, tau2=2):
    correct_CMI_forw = 0
    wrong_CMI_forw = 0
    correct_CMI_back = 0
    wrong_CMI_back = 0
    correct_TE_forw = 0
    wrong_TE_forw = 0
    correct_TE_back = 0
    wrong_TE_back = 0
    correct_fullCI = 0
    correct_PCMCI = 0
    wrong_fullCI = 0
    wrong_PCMCI = 0
    
    tot_links = 0
    correct_links = set([0,1,4])
    
    res = {
                "delta" : [], # list with all deltas
                "numSelected" : [], #
                "selectedFeatures" : [] 
            #    "accuracy" : [] # list of scores associated with the reduced problem
            }
    
    for seed in range(n_reps):
        
        ### generate dataset
        x,y = generate_dataset_5dim(data_for_X1=data_for_X1, data_for_X3=data_for_X3, a=a, b=b, c=c, n=n, seed=seed, noise=noise)
        
        k = round(n/15)
        
        sel = forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_forw_sel = set(sel)
        
        sel = TE_forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau, tau2=tau2)
        print(sel)
        TE_forw_sel = set(sel)
        
        sel = backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_back_sel = set(sel)
    
        sel = TE_backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau, tau2=tau2)
        print(sel)
        TE_back_sel = set(sel)
        
        p = run_fullCI(pd.DataFrame(np.concatenate((x,y),axis=1), columns = ['x0','x1','x2','x3','y']),tau=tau,cond_test=ParCorr())
        sel = []
        for i in [0,1,2,3,4]:
            if min(p[i,4,:])<0.05:
                sel.append(i)
        full_CI_sel = set(sel)
        print(full_CI_sel)
        
        p = run_PCMCI(pd.DataFrame(np.concatenate((x,y),axis=1), columns = ['x0','x1','x2','x3','y']),tau=tau,cond_test=ParCorr())
        sel = []
        for i in [0,1,2,3,4]:
            if min(p[i,4,:])<0.05:
                sel.append(i)
        PCMCI_sel = set(sel)
        print(PCMCI_sel)
        
        print('\n\n')
        
        correct_CMI_forw += len([found for found in CMI_forw_sel if found in correct_links])
        wrong_CMI_forw += len(CMI_forw_sel) - len([found for found in CMI_forw_sel if found in correct_links])
        correct_CMI_back += len([found for found in CMI_back_sel if found in correct_links])
        wrong_CMI_back += len(CMI_back_sel) - len([found for found in CMI_back_sel if found in correct_links])
        correct_TE_forw += len([found for found in TE_forw_sel if found in correct_links])
        wrong_TE_forw += len(TE_forw_sel) - len([found for found in TE_forw_sel if found in correct_links])
        correct_TE_back += len([found for found in TE_back_sel if found in correct_links])
        wrong_TE_back += len(TE_back_sel) - len([found for found in TE_back_sel if found in correct_links])
        correct_fullCI += len([found for found in full_CI_sel if found in correct_links])
        wrong_fullCI += len(full_CI_sel) - len([found for found in full_CI_sel if found in correct_links])
        correct_PCMCI += len([found for found in PCMCI_sel if found in correct_links])
        wrong_PCMCI += len(PCMCI_sel) - len([found for found in PCMCI_sel if found in correct_links])
        tot_links += len(correct_links)
        
        #compute_r2([x[:,0],x[:,1]],y,tau=2,n=300)
    
    corrects = [correct_CMI_forw, correct_CMI_back, correct_TE_forw, correct_TE_back,correct_fullCI,correct_PCMCI]
    wrongs = [wrong_CMI_forw, wrong_CMI_back, wrong_TE_forw, wrong_TE_back, wrong_fullCI, wrong_PCMCI]
    print(corrects)
    print(wrongs)  
    print(tot_links)

    return corrects, wrongs, tot_links


In [ ]:
def generate_dataset_5dim(data_for_X1, data_for_X3, a, b, c, n, seed, noise):
    
    data_for_X1 = pd.read_csv('../causalFS/data/sampleX0.csv')
    data_for_X3 = pd.read_csv('../causalFS/data/sampleX1.csv')
    rng = np.random.default_rng(seed=seed)
    random.seed(seed)
    np.random.seed(seed)
    
    x1 = data_for_X1.iloc[:n+2].values
    x1 +=  np.random.randn(len(x1),1) * noise
    plt.plot(x1[2:],'g')
    
    x3 = data_for_X3.iloc[:n+2].values
    x3 +=  np.random.randn(len(x3),1) * noise
    plt.plot(x3[2:],'y')
    
    x0 = list(np.zeros(n+2))
    for i in range(n):
        next_val = x3[i+1]*a + np.random.randn() * noise
        x0[i+2] = next_val[0] 
    plt.plot(x0[2:],'r')
    
    y = list(np.zeros(n+2))
    for i in range(n):
        next_val = y[i]*c[0] + x0[i+1]*c[1] + x1[i]*c[2] + np.random.randn() * noise
        y[i+2] = next_val[0] 
    plt.plot(y[2:],'b')
    
    x2 = list(np.zeros(n+2))
    for i in range(n):
        next_val = y[i+1]*b + np.random.randn() * noise
        x2[i+2] = next_val
    plt.plot(x2[2:],'brown')
    plt.show()
    
    x0 = np.array(x0)[2:].reshape(-1,1)
    x1 = np.array(x1)[2:].reshape(-1,1)
    x2 = np.array(x2)[2:].reshape(-1,1)
    x3 = np.array(x3)[2:].reshape(-1,1)
    y = np.array(y)[2:].reshape(-1,1)
    print(x0.shape, x1.shape,x2.shape, x3.shape, y.shape)

    x = np.concatenate((x0,x1,x2,x3),axis=1)
    
    return x,y



### single example

In [ ]:
corrects,wrongs,tot_links = experiment_5dim_fixedCoeff(data_for_X1='', data_for_X3='', a=0.62, b=-0.85, c=[-0.97,0.63,0.83], n=300, noise=0.1, n_reps=10, tau=2)


In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### increasing noise

In [ ]:

corrects=[]
wrongs = []
tot_links = []
for noise in [0.01,0.05,0.1,0.15,0.2]:
    corr, wron, tot_li = experiment_5dim_fixedCoeff(data_for_X1='', data_for_X3='', a=0.62, b=-0.85, c=[-0.97,0.63,0.83], n=300, noise=noise, n_reps=10, tau=2)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)


In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### variable number of samples

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []
for n in [100,200,300,400,500]:
    corr, wron, tot_li = experiment_5dim_fixedCoeff(data_for_X1='', data_for_X3='', a=0.62, b=-0.85, c=[-0.97,0.63,0.83], n=n, noise=0.1, n_reps=10, tau=2)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)


In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### variable coefficients

In [ ]:
rng = np.random.default_rng(seed=0)
random.seed(0)
np.random.seed(0)
cc = []
aa = []
bb = []
while len(cc)<15:
    comb = [rng.uniform(-1,1),rng.uniform(-1,1),rng.uniform(-1,1)]
    if (((comb[0]<=-0.5) | (comb[0]>=0.5)) & ((comb[1]<=-0.5) | (comb[1]>=0.5)) & ((comb[2]<=-0.5) | (comb[2]>=0.5))):
        cc.append(comb)
        
while len(aa)<15:
    val = rng.uniform(-1,1)
    if (((val<=-0.5) | (val>=0.5))):
        aa.append(val)
        
while len(bb)<15:
    val = rng.uniform(-1,1)
    if (((val<=-0.5) | (val>=0.5))):
        bb.append(val)
    
print(aa,bb,cc)



In [ ]:
corrects=[]
wrongs = [] 
tot_links = []

for i in [0,3,6,9,12]:
    corr, wron, tot_li = experiment_5dim_fixedCoeff(data_for_X1='', data_for_X3='', a=aa[i], b=bb[i], c=cc[i], n=300, noise=0.1, n_reps=10, tau=2)

    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)

In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### variable tau

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []
for tau in [2,3,4,5]:
    corr, wron, tot_li = experiment_5dim_fixedCoeff(data_for_X1='', data_for_X3='', a=0.62, b=-0.85, c=[-0.97,0.63,0.83], n=300, noise=0.01, n_reps=10, tau=tau)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)

In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

# Same, 10 variables

### Preliminaries

In [ ]:
def generate_dataset_10dim(k, n, seed, noise):
    
    data_for_X0 = pd.read_csv('../causalFS/data/sampleX0.csv')
    data_for_X5 = pd.read_csv('../causalFS/data/sampleX2.csv')
    data_for_X7 = pd.read_csv('../causalFS/data/sampleX3.csv')

    rng = np.random.default_rng(seed=seed)
    random.seed(seed)
    np.random.seed(seed)
    
    ############## independent variables: x0,x5,x7 #################
    ### X0
    x0 = data_for_X0.iloc[:n+3].values
    x0 +=  np.random.randn(len(x0),1) * noise
    plt.plot(x0[3:],'r')
    
    ### X5
    x5 = data_for_X5.iloc[:n+3].values
    x5 +=  np.random.randn(len(x5),1) * noise
    plt.plot(x5[3:],'g')
    
    ### X7
    x7 = data_for_X7.iloc[:n+3].values
    x7 +=  np.random.randn(len(x7),1) * noise
    plt.plot(x7[3:],'b')
    plt.show()
    
    ############## first order dependence: x4,x8 #################
    
    x4 = list(np.zeros(n+3))
    for i in range(n):
        next_val = x0[i+2]*k[0] + np.random.randn() * noise
        x4[i+3] = next_val[0] 
    plt.plot(x4[3:],'r')
    
    x8 = list(np.zeros(n+3))
    for i in range(n):
        next_val = x7[i+2]*k[1] + np.random.randn() * noise
        x8[i+3] = next_val[0] 
    plt.plot(x8[3:],'g')
    plt.show()
    
    ############## second order dependence: x3,y #################
    
    x3 = list(np.zeros(n+3))
    for i in range(n):
        next_val = x0[i+1]*k[2] + x3[i+2]*k[3] + np.random.randn() * noise
        x3[i+3] = next_val[0]
    plt.plot(x3[3:],'r')
    
    y = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+1]*k[6] + x5[i]*k[5] + x8[i+2]*k[4] + np.random.randn() * noise
        y[i+3] = next_val[0] 
    plt.plot(y[3:],'b')
    plt.show()
    
    ############## third order dependence: x3,y #################
    
    x6 = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+2]*k[7] + np.random.randn() * noise
        x6[i+3] = next_val
    plt.plot(x6[3:],'r')
    
    x1 = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+1]*k[8] + np.random.randn() * noise
        x1[i+3] = next_val
    plt.plot(x1[3:],'g')
    
    x2 = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+2]*k[9] + np.random.randn() * noise
        x2[i+3] = next_val
    plt.plot(x2[3:],'b')
    plt.show()
    
    x0 = np.array(x0)[3:].reshape(-1,1)
    x1 = np.array(x1)[3:].reshape(-1,1)
    x2 = np.array(x2)[3:].reshape(-1,1)
    x3 = np.array(x3)[3:].reshape(-1,1)
    x4 = np.array(x4)[3:].reshape(-1,1)
    x5 = np.array(x5)[3:].reshape(-1,1)
    x6 = np.array(x6)[3:].reshape(-1,1)
    x7 = np.array(x7)[3:].reshape(-1,1)
    x8 = np.array(x8)[3:].reshape(-1,1)
    y = np.array(y)[3:].reshape(-1,1)
    print(x0.shape, x1.shape,x2.shape, x3.shape, x4.shape, x5.shape, x6.shape, x7.shape, x8.shape, y.shape)

    x = np.concatenate((x0,x1,x2,x3,x4,x5,x6,x7,x8),axis=1)
    
    return x,y



In [ ]:
def experiment_10dim_fixedCoeff(coeff, n, noise=0.1, n_reps=10, tau=3, tau2=3):
    correct_CMI_forw = 0
    wrong_CMI_forw = 0
    correct_CMI_back = 0
    wrong_CMI_back = 0
    correct_TE_forw = 0
    wrong_TE_forw = 0
    correct_TE_back = 0
    wrong_TE_back = 0
    correct_fullCI = 0
    correct_PCMCI = 0
    wrong_fullCI = 0
    wrong_PCMCI = 0
    
    tot_links = 0
    correct_links = set([5,8,9])
    
    res = {
                "delta" : [], # list with all deltas
                "numSelected" : [], #
                "selectedFeatures" : [] 
            #    "accuracy" : [] # list of scores associated with the reduced problem
            }
    
    for seed in range(n_reps):
        
        ### generate dataset
        x,y = generate_dataset_10dim(coeff, n, seed, noise)
        
        k = round(n/15)
        
        sel = forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_forw_sel = set(sel)
        
        sel = TE_forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau, tau2=tau2)
        print(sel)
        TE_forw_sel = set(sel)
        
        sel = backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_back_sel = set(sel)
    
        sel = TE_backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau, tau2=tau2)
        print(sel)
        TE_back_sel = set(sel)
        
        p = run_fullCI(pd.DataFrame(np.concatenate((x,y),axis=1), columns = ['x0','x1','x2','x3','x4','x5','x6','x7','x8','y']),tau=tau,cond_test=ParCorr())
        sel = []
        for i in [0,1,2,3,4,5,6,7,8,9]:
            if min(p[i,9,:])<0.05:
                sel.append(i)
        full_CI_sel = set(sel)
        print(full_CI_sel)
        
        p = run_PCMCI(pd.DataFrame(np.concatenate((x,y),axis=1), columns = ['x0','x1','x2','x3','x4','x5','x6','x7','x8','y']),tau=tau,cond_test=ParCorr())
        sel = []
        for i in [0,1,2,3,4,5,6,7,8,9]:
            if min(p[i,9,:])<0.05:
                sel.append(i)
        PCMCI_sel = set(sel)
        print(PCMCI_sel)
        
        print('\n\n')
        
        correct_CMI_forw += len([found for found in CMI_forw_sel if found in correct_links])
        wrong_CMI_forw += len(CMI_forw_sel) - len([found for found in CMI_forw_sel if found in correct_links])
        correct_CMI_back += len([found for found in CMI_back_sel if found in correct_links])
        wrong_CMI_back += len(CMI_back_sel) - len([found for found in CMI_back_sel if found in correct_links])
        correct_TE_forw += len([found for found in TE_forw_sel if found in correct_links])
        wrong_TE_forw += len(TE_forw_sel) - len([found for found in TE_forw_sel if found in correct_links])
        correct_TE_back += len([found for found in TE_back_sel if found in correct_links])
        wrong_TE_back += len(TE_back_sel) - len([found for found in TE_back_sel if found in correct_links])
        correct_fullCI += len([found for found in full_CI_sel if found in correct_links])
        wrong_fullCI += len(full_CI_sel) - len([found for found in full_CI_sel if found in correct_links])
        correct_PCMCI += len([found for found in PCMCI_sel if found in correct_links])
        wrong_PCMCI += len(PCMCI_sel) - len([found for found in PCMCI_sel if found in correct_links])
        tot_links += len(correct_links)
        
        #compute_r2([x[:,0],x[:,1]],y,tau=2,n=300)
    
    corrects = [correct_CMI_forw, correct_CMI_back, correct_TE_forw, correct_TE_back,correct_fullCI,correct_PCMCI]
    wrongs = [wrong_CMI_forw, wrong_CMI_back, wrong_TE_forw, wrong_TE_back, wrong_fullCI, wrong_PCMCI]
    print(corrects)
    print(wrongs)  
    print(tot_links)

    return corrects, wrongs, tot_links


### single experiment

In [ ]:
corrects, wrongs, tot_links = experiment_10dim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3)

In [ ]:
corrects

In [ ]:
wrongs

### increasing noise

In [ ]:
corrects=[]
wrongs = []
tot_links = []
for noise in [0.01,0.05,0.1,0.15,0.2]:
    corr, wron, tot_li = experiment_10dim_fixedCoeff(coeffs[2], n=300, noise=noise, n_reps=10, tau=3, tau2=3)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)


In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### variable number of samples

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []
for n in [100,200,300,400,500]:
    corr, wron, tot_li = experiment_10dim_fixedCoeff(coeffs[2], n=n, noise=0.1, n_reps=10, tau=3, tau2=3)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)


In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### variable tau

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []
for tau in [2,3,4,5]:
    corr, wron, tot_li = experiment_10dim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=tau, tau2=tau)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)


In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

### variable coefficients

In [ ]:
rng = np.random.default_rng(seed=0)
random.seed(0)
np.random.seed(0)
coeffs = []

for i in range(10):
    cc=[]
    while len(cc)<10:
        val = rng.uniform(-1,1)
        if (((val<=-0.5) | (val>=0.5))):
            cc.append(val)
    coeffs.append(cc)
print(coeffs)



In [ ]:
coeffs[2]

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []

for i in range(5):
    corr, wron, tot_li = experiment_10dim_fixedCoeff(coeff=coeffs[i+2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)

In [ ]:
corrects=[]
wrongs = [] 
tot_links = []

for i in range(5):
    corr, wron, tot_li = experiment_10dim_fixedCoeff(coeff=coeffs[i+2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3)
    corrects.append(corr)
    wrongs.append(wron)
    tot_links.append(tot_li)

In [ ]:
corrects

In [ ]:
corrects

In [ ]:
[[20, 20, 20, 20, 30, 29],
 [19, 19, 4, 8, 30, 23],
 [20, 20, 0, 19, 30, 26],
 [11, 11, 10, 10, 30, 29],
 [20, 20, 20, 20, 30, 28]]

In [ ]:
wrongs

In [ ]:
tot_links

# 15 variables

In [ ]:
def generate_dataset_15dim(k, n, seed, noise):
    
    data_for_X0 = pd.read_csv('../causalFS/data/sampleX0.csv')
    data_for_X5 = pd.read_csv('../causalFS/data/sampleX2.csv')
    data_for_X7 = pd.read_csv('../causalFS/data/sampleX3.csv')

    rng = np.random.default_rng(seed=seed)
    random.seed(seed)
    np.random.seed(seed)
    
    ############## independent variables: x0,x5,x7 #################
    ### X0
    x0 = data_for_X0.iloc[:n+3].values
    x0 +=  np.random.randn(len(x0),1) * noise
    plt.plot(x0[3:],'r')
    
    ### X5
    x5 = data_for_X5.iloc[:n+3].values
    x5 +=  np.random.randn(len(x5),1) * noise
    plt.plot(x5[3:],'g')
    
    ### X7
    x7 = data_for_X7.iloc[:n+3].values
    x7 +=  np.random.randn(len(x7),1) * noise
    plt.plot(x7[3:],'b')
    plt.show()
    
    ############## first order dependence: x4,x8 #################
    
    x4 = list(np.zeros(n+3))
    for i in range(n):
        next_val = x0[i+2]*k[0] + np.random.randn() * noise
        x4[i+3] = next_val[0] 
    plt.plot(x4[3:],'r')
    
    x8 = list(np.zeros(n+3))
    for i in range(n):
        next_val = x7[i+2]*k[1] + np.random.randn() * noise
        x8[i+3] = next_val[0] 
    plt.plot(x8[3:],'g')
    plt.show()
    
    ############## second order dependence: x3,y #################
    
    x3 = list(np.zeros(n+3))
    for i in range(n):
        next_val = x0[i+1]*k[2] + x3[i+2]*k[3] + np.random.randn() * noise
        x3[i+3] = next_val[0]
    plt.plot(x3[3:],'r')
    
    y = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+1]*k[6] + x5[i]*k[5] + x8[i+2]*k[4] + np.random.randn() * noise
        y[i+3] = next_val[0] 
    plt.plot(y[3:],'b')
    plt.show()
    
    ############## third order dependence: x3,y #################
    
    x6 = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+2]*k[7] + np.random.randn() * noise
        x6[i+3] = next_val
    plt.plot(x6[3:],'r')
    
    x1 = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+1]*k[8] + np.random.randn() * noise
        x1[i+3] = next_val
    plt.plot(x1[3:],'g')
    
    x2 = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+2]*k[9] + np.random.randn() * noise
        x2[i+3] = next_val
    plt.plot(x2[3:],'b')
    plt.show()
    
    x0 = np.array(x0)[3:].reshape(-1,1)
    x1 = np.array(x1)[3:].reshape(-1,1)
    x2 = np.array(x2)[3:].reshape(-1,1)
    x3 = np.array(x3)[3:].reshape(-1,1)
    x4 = np.array(x4)[3:].reshape(-1,1)
    x5 = np.array(x5)[3:].reshape(-1,1)
    x6 = np.array(x6)[3:].reshape(-1,1)
    x7 = np.array(x7)[3:].reshape(-1,1)
    x8 = np.array(x8)[3:].reshape(-1,1)
    y = np.array(y)[3:].reshape(-1,1)
    print(x0.shape, x1.shape,x2.shape, x3.shape, x4.shape, x5.shape, x6.shape, x7.shape, x8.shape, y.shape)

    x = np.concatenate((x0,x1,x2,x3,x4,x5,x6,x7,x8),axis=1)
    
    for i in range(2):
        aux_x = np.random.randn(n+3)
        plt.plot(aux_x[3:],'b')
        
        aux_x2 = list(np.zeros(n+3))
        for i in range(n):
            next_val = aux_x[i+2] * rng.uniform(-1,1) + np.random.randn() * noise
            aux_x2[i+3] = next_val
        plt.plot(aux_x2[3:],'r')
        
        aux_x3 = list(np.zeros(n+3))
        for i in range(n):
            next_val = aux_x2[i+1] * rng.uniform(-1,1) + aux_x3[i] * rng.uniform(-1,1) + np.random.randn() * noise
            aux_x3[i+3] = next_val
        plt.plot(aux_x3[3:],'g')
        x = np.concatenate((x,aux_x[3:].reshape(-1,1),np.array(aux_x2)[3:].reshape(-1,1),np.array(aux_x3)[3:].reshape(-1,1)),axis=1)
        plt.show()
    
    return x,y



In [ ]:
def experiment_15dim_fixedCoeff(coeff, n, noise=0.1, n_reps=10, tau=3, tau2=3):
    correct_CMI_forw = 0
    wrong_CMI_forw = 0
    correct_CMI_back = 0
    wrong_CMI_back = 0
    correct_TE_forw = 0
    wrong_TE_forw = 0
    correct_TE_back = 0
    wrong_TE_back = 0
    correct_fullCI = 0
    correct_PCMCI = 0
    wrong_fullCI = 0
    wrong_PCMCI = 0
    
    tot_links = 0
    correct_links = set([5,8,9])
    
    res = {
                "delta" : [], # list with all deltas
                "numSelected" : [], #
                "selectedFeatures" : [] 
            #    "accuracy" : [] # list of scores associated with the reduced problem
            }
    
    for seed in range(n_reps):
        
        ### generate dataset
        x,y = generate_dataset_15dim(coeff, n, seed, noise)
        
        k = round(n/15)
        
        sel = forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_forw_sel = set(sel)
        
        sel = TE_forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau, tau2=tau2)
        print(sel)
        TE_forw_sel = set(sel)
        
        sel = backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_back_sel = set(sel)
    
        sel = TE_backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau, tau2=tau2)
        print(sel)
        TE_back_sel = set(sel)
        
        p = run_fullCI(pd.DataFrame(np.concatenate((x,y),axis=1), columns = ['x0','x1','x2','x3','x4','x5','x6','x7','x8','y','x9','x10','x11','x12','x13','x14']),tau=tau,cond_test=ParCorr())
        sel = []
        for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
            if min(p[i,9,:])<0.05:
                sel.append(i)
        full_CI_sel = set(sel)
        print(full_CI_sel)
        
        p = run_PCMCI(pd.DataFrame(np.concatenate((x,y),axis=1), columns = ['x0','x1','x2','x3','x4','x5','x6','x7','x8','y','x9','x10','x11','x12','x13','x14']),tau=tau,cond_test=ParCorr())
        sel = []
        for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
            if min(p[i,9,:])<0.05:
                sel.append(i)
        PCMCI_sel = set(sel)
        print(PCMCI_sel)
        
        print('\n\n')
        
        correct_CMI_forw += len([found for found in CMI_forw_sel if found in correct_links])
        wrong_CMI_forw += len(CMI_forw_sel) - len([found for found in CMI_forw_sel if found in correct_links])
        correct_CMI_back += len([found for found in CMI_back_sel if found in correct_links])
        wrong_CMI_back += len(CMI_back_sel) - len([found for found in CMI_back_sel if found in correct_links])
        correct_TE_forw += len([found for found in TE_forw_sel if found in correct_links])
        wrong_TE_forw += len(TE_forw_sel) - len([found for found in TE_forw_sel if found in correct_links])
        correct_TE_back += len([found for found in TE_back_sel if found in correct_links])
        wrong_TE_back += len(TE_back_sel) - len([found for found in TE_back_sel if found in correct_links])
        correct_fullCI += len([found for found in full_CI_sel if found in correct_links])
        wrong_fullCI += len(full_CI_sel) - len([found for found in full_CI_sel if found in correct_links])
        correct_PCMCI += len([found for found in PCMCI_sel if found in correct_links])
        wrong_PCMCI += len(PCMCI_sel) - len([found for found in PCMCI_sel if found in correct_links])
        tot_links += len(correct_links)
        
        #compute_r2([x[:,0],x[:,1]],y,tau=2,n=300)
    
    corrects = [correct_CMI_forw, correct_CMI_back, correct_TE_forw, correct_TE_back,correct_fullCI,correct_PCMCI]
    wrongs = [wrong_CMI_forw, wrong_CMI_back, wrong_TE_forw, wrong_TE_back, wrong_fullCI, wrong_PCMCI]
    print(corrects)
    print(wrongs)  
    print(tot_links)

    return corrects, wrongs, tot_links


In [ ]:
coeffs[2]

In [ ]:
corrects, wrongs, tot_links = experiment_15dim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3)

In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

In [ ]:
### others: 
15*10-30

# N variables

In [ ]:
def generate_dataset_Ndim(k, n, seed, noise, n_add=2):
    
    data_for_X0 = pd.read_csv('../causalFS/data/sampleX0.csv')
    data_for_X5 = pd.read_csv('../causalFS/data/sampleX2.csv')
    data_for_X7 = pd.read_csv('../causalFS/data/sampleX3.csv')

    rng = np.random.default_rng(seed=seed)
    random.seed(seed)
    np.random.seed(seed)
    
    ############## independent variables: x0,x5,x7 #################
    ### X0
    x0 = data_for_X0.iloc[:n+3].values
    x0 +=  np.random.randn(len(x0),1) * noise
    plt.plot(x0[3:],'r')
    
    ### X5
    x5 = data_for_X5.iloc[:n+3].values
    x5 +=  np.random.randn(len(x5),1) * noise
    plt.plot(x5[3:],'g')
    
    ### X7
    x7 = data_for_X7.iloc[:n+3].values
    x7 +=  np.random.randn(len(x7),1) * noise
    plt.plot(x7[3:],'b')
    plt.show()
    
    ############## first order dependence: x4,x8 #################
    
    x4 = list(np.zeros(n+3))
    for i in range(n):
        next_val = x0[i+2]*k[0] + np.random.randn() * noise
        x4[i+3] = next_val[0] 
    plt.plot(x4[3:],'r')
    
    x8 = list(np.zeros(n+3))
    for i in range(n):
        next_val = x7[i+2]*k[1] + np.random.randn() * noise
        x8[i+3] = next_val[0] 
    plt.plot(x8[3:],'g')
    plt.show()
    
    ############## second order dependence: x3,y #################
    
    x3 = list(np.zeros(n+3))
    for i in range(n):
        next_val = x0[i+1]*k[2] + x3[i+2]*k[3] + np.random.randn() * noise
        x3[i+3] = next_val[0]
    plt.plot(x3[3:],'r')
    
    y = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+1]*k[6] + x5[i]*k[5] + x8[i+2]*k[4] + np.random.randn() * noise
        y[i+3] = next_val[0] 
    plt.plot(y[3:],'b')
    plt.show()
    
    ############## third order dependence: x3,y #################
    
    x6 = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+2]*k[7] + np.random.randn() * noise
        x6[i+3] = next_val
    plt.plot(x6[3:],'r')
    
    x1 = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+1]*k[8] + np.random.randn() * noise
        x1[i+3] = next_val
    plt.plot(x1[3:],'g')
    
    x2 = list(np.zeros(n+3))
    for i in range(n):
        next_val = y[i+2]*k[9] + np.random.randn() * noise
        x2[i+3] = next_val
    plt.plot(x2[3:],'b')
    plt.show()
    
    x0 = np.array(x0)[3:].reshape(-1,1)
    x1 = np.array(x1)[3:].reshape(-1,1)
    x2 = np.array(x2)[3:].reshape(-1,1)
    x3 = np.array(x3)[3:].reshape(-1,1)
    x4 = np.array(x4)[3:].reshape(-1,1)
    x5 = np.array(x5)[3:].reshape(-1,1)
    x6 = np.array(x6)[3:].reshape(-1,1)
    x7 = np.array(x7)[3:].reshape(-1,1)
    x8 = np.array(x8)[3:].reshape(-1,1)
    y = np.array(y)[3:].reshape(-1,1)
    print(x0.shape, x1.shape,x2.shape, x3.shape, x4.shape, x5.shape, x6.shape, x7.shape, x8.shape, y.shape)

    x = np.concatenate((x0,x1,x2,x3,x4,x5,x6,x7,x8),axis=1)
    
    for i in range(n_add):
        aux_x = np.random.randn(n+3)
        plt.plot(aux_x[3:],'b')
        
        aux_x2 = list(np.zeros(n+3))
        for i in range(n):
            next_val = aux_x[i+2] * rng.uniform(-1,1) + np.random.randn() * noise
            aux_x2[i+3] = next_val
        plt.plot(aux_x2[3:],'r')
        
        aux_x3 = list(np.zeros(n+3))
        for i in range(n):
            next_val = aux_x2[i+1] * rng.uniform(-1,1) + aux_x3[i] * rng.uniform(-1,1) + np.random.randn() * noise
            aux_x3[i+3] = next_val
        plt.plot(aux_x3[3:],'g')
        x = np.concatenate((x,aux_x[3:].reshape(-1,1),np.array(aux_x2)[3:].reshape(-1,1),np.array(aux_x3)[3:].reshape(-1,1)),axis=1)
        plt.show()
    
    return x,y



In [ ]:
def experiment_Ndim_fixedCoeff(coeff, n, noise=0.1, n_reps=10, tau=3, tau2=3, n_add=2):
    correct_CMI_forw = 0
    wrong_CMI_forw = 0
    correct_CMI_back = 0
    wrong_CMI_back = 0
    correct_TE_forw = 0
    wrong_TE_forw = 0
    correct_TE_back = 0
    wrong_TE_back = 0
    correct_fullCI = 0
    correct_PCMCI = 0
    wrong_fullCI = 0
    wrong_PCMCI = 0
    
    tot_links = 0
    correct_links = set([5,8,9])
    
    res = {
                "delta" : [], # list with all deltas
                "numSelected" : [], #
                "selectedFeatures" : [] 
            #    "accuracy" : [] # list of scores associated with the reduced problem
            }
    
    for seed in range(n_reps):
        
        ### generate dataset
        x,y = generate_dataset_Ndim(coeff, n, seed, noise, n_add)
        print(f'####### Dimension:{x.shape[1]} ############')
        k = round(n/15)
        
        sel = forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_forw_sel = set(sel)
        
        sel = TE_forwardFeatureSelection(threshold=100,features=x,target=y,res=res,k=k, nproc=1, tau=tau, tau2=tau2)
        print(sel)
        TE_forw_sel = set(sel)
        
        sel = backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau)
        print(sel)
        CMI_back_sel = set(sel)
    
        sel = TE_backwardFeatureSelection(threshold=0.000001,features=x,target=y,res=res,k=k, nproc=1, tau=tau, tau2=tau2)
        print(sel)
        TE_back_sel = set(sel)
        
        p = run_fullCI(pd.DataFrame(np.concatenate((x,y),axis=1)),tau=tau,cond_test=ParCorr())
        sel = []
        for i in range(x.shape[1]+1):
            if min(p[i,-1,:])<0.05:
                sel.append(i)
        full_CI_sel = set(sel)
        print(full_CI_sel)
        
        p = run_PCMCI(pd.DataFrame(np.concatenate((x,y),axis=1)),tau=tau,cond_test=ParCorr())
        sel = []
        for i in range(x.shape[1]+1):
            if min(p[i,-1,:])<0.05:
                sel.append(i)
        PCMCI_sel = set(sel)
        print(PCMCI_sel)
        
        print('\n\n')
        
        correct_CMI_forw += len([found for found in CMI_forw_sel if found in correct_links])
        wrong_CMI_forw += len(CMI_forw_sel) - len([found for found in CMI_forw_sel if found in correct_links])
        correct_CMI_back += len([found for found in CMI_back_sel if found in correct_links])
        wrong_CMI_back += len(CMI_back_sel) - len([found for found in CMI_back_sel if found in correct_links])
        correct_TE_forw += len([found for found in TE_forw_sel if found in correct_links])
        wrong_TE_forw += len(TE_forw_sel) - len([found for found in TE_forw_sel if found in correct_links])
        correct_TE_back += len([found for found in TE_back_sel if found in correct_links])
        wrong_TE_back += len(TE_back_sel) - len([found for found in TE_back_sel if found in correct_links])
        correct_fullCI += len([found for found in full_CI_sel if found in correct_links])
        wrong_fullCI += len(full_CI_sel) - len([found for found in full_CI_sel if found in correct_links])
        correct_PCMCI += len([found for found in PCMCI_sel if found in correct_links])
        wrong_PCMCI += len(PCMCI_sel) - len([found for found in PCMCI_sel if found in correct_links])
        tot_links += len(correct_links)
        
        #compute_r2([x[:,0],x[:,1]],y,tau=2,n=300)
    
    corrects = [correct_CMI_forw, correct_CMI_back, correct_TE_forw, correct_TE_back,correct_fullCI,correct_PCMCI]
    wrongs = [wrong_CMI_forw, wrong_CMI_back, wrong_TE_forw, wrong_TE_back, wrong_fullCI, wrong_PCMCI]
    print(corrects)
    print(wrongs)  
    print(tot_links)

    return corrects, wrongs, tot_links


In [ ]:
corrects, wrongs, tot_links = experiment_Ndim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3, n_add=2)



In [ ]:
corrects

In [ ]:
wrongs

In [ ]:
tot_links

In [ ]:
corrects, wrongs, tot_links = experiment_Ndim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3, n_add=3)


In [ ]:
print(f'{corrects},\n{wrongs},\n{tot_links}')

In [ ]:
corrects, wrongs, tot_links = experiment_Ndim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3, n_add=7)


In [ ]:
print(f'{corrects},\n{wrongs},\n{tot_links}')

In [ ]:
corrects, wrongs, tot_links = experiment_Ndim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3, n_add=10)


In [ ]:
print(f'{corrects},\n{wrongs},\n{tot_links}')

In [ ]:
corrects, wrongs, tot_links = experiment_Ndim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3, n_add=17)


In [ ]:
print(f'{corrects},\n{wrongs},\n{tot_links}')

In [ ]:
corrects, wrongs, tot_links = experiment_Ndim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3, n_add=23)


In [ ]:
print(f'{corrects},\n{wrongs},\n{tot_links}')

In [ ]:
corrects, wrongs, tot_links = experiment_Ndim_fixedCoeff(coeffs[2], n=300, noise=0.1, n_reps=10, tau=3, tau2=3, n_add=30)


In [ ]:
print(f'{corrects},\n{wrongs},\n{tot_links}')